# Zero shot feelings and needs guesses

In [ ]:
import importlib

import numpy as np
import tensorflow as tf

from transformers import pipeline

from howie import feelings, needs

In [ ]:
#boiler plate for limiting allocated GPU memory
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)


# Guessing emotions and needs

In [ ]:
classifier = pipeline("zero-shot-classification", device=0)

In [ ]:
sequence = "I haven't eaten all day"
hypothesis_template = "I feel {}."
results = classifier(sequence, feelings.base_feelings, hypothesis_template=hypothesis_template,multi_label=True)
results['labels'][:5]

In [ ]:
sequence = "I aced the interview and got the job."
hypothesis_template = "I feel {}."
results = classifier(sequence, feelings.nuanced_feelings, hypothesis_template=hypothesis_template,multi_label=True)
results['labels'][:5]

In [ ]:
sequence = "I aced the interview and got the job."
hypothesis_template = "I feel {}."
results = classifier(sequence, needs.needs, hypothesis_template=hypothesis_template,multi_label=True)
results['labels'][:5]

In [ ]:
sequence = "You're abandoning me."
hypothesis_template = "I feel {}."
results = classifier(sequence, feelings.base_feelings, hypothesis_template=hypothesis_template,multi_label=True)
results['labels'][:5]